In [ ]:
import numpy as np
import spacy
import re
import nltk
import pickle
import json
import pandas as pd
from spacy.lang.nb import Norwegian
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.linear_model import SGDClassifier

# nltk.download("stopwords")
# nltk.download("wordnet")

In [2]:
!py -m spacy download nb_core_news_sm

In [3]:
data = {}
stops = stopwords.words('norwegian')
nlp = spacy.load('nb_core_news_sm')

# print(stops)

for name in ["train", "dev", "test"]:
    with open("{0}.json".format(name)) as infile:
        data[name] = json.load(infile)

In [4]:
train = pd.DataFrame.from_dict(data["train"])
test = pd.DataFrame.from_dict(data["test"])

X_train = train['text']
y_train = train['label']

X_test = test['text']
y_test = test['label']

# train
# test

In [5]:
stemmer = nlp.get_pipe('lemmatizer')

def lemmatize(X: list[str], n_to_print=0):
    documents = []
    for sen in range(0, len(X)):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(X[sen]))

        if len(documents) < n_to_print:
            print('Original:', document)

        # remove all single characters
        document = re.sub(r'\s+[a-zA-ZæøåÆØÅ]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-ZæøåÆØÅ]\s+', ' ', document)

        if len(documents) < n_to_print:
            print('Removed single characters:', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        # document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        if len(documents) < n_to_print:
            print('Lowercase:', document)

        # Lemmatization
        document = nlp(document)

        document = [word.lemma_ for word in document]

        if len(documents) < n_to_print:
            print('Lemmatized:', document)
        document = ' '.join(document)

        documents.append(document)
    return documents


In [6]:
X_train = lemmatize(X_train, 2)
X_test = lemmatize(X_test)

vectorizer = TfidfVectorizer(stop_words=stops, ngram_range=(1,2))

X_train = vectorizer.fit_transform(X_train).toarray()

X_train

Original: Philips 190G6
Removed single characters: Philips 190G6
Lowercase: philips 190g6
Lemmatized: ['philips', '190g6']
Original: Med integrerte høyttalere som på ingen måte er diskret plassert   og med en stor subwoofer inkludert   da snakker vi om en gutteskjerm  
Removed single characters: Med integrerte høyttalere som på ingen måte er diskret plassert   og med en stor subwoofer inkludert   da snakker vi om en gutteskjerm  
Lowercase: med integrerte høyttalere som på ingen måte er diskret plassert og med en stor subwoofer inkludert da snakker vi om en gutteskjerm 
Lemmatized: ['med', 'integrert', 'høyttal', 'som', 'på', 'ingen', 'måte', 'være', 'diskre', 'plassere', 'og', 'med', 'en', 'stor', 'subwoof', 'inkludere', 'da', 'snakke', 'vi', 'om', 'en', 'gutteskjerm']


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
X_test = vectorizer.transform(X_test).toarray()

X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
classifierM = SGDClassifier()
classifierM.fit(X_train, y_train)
y_predM = classifierM.predict(X_test)

In [ ]:
print(f'MultinomialNB + TFID: {accuracy_score(y_test, y_predM)}')

In [ ]:
print(confusion_matrix(y_test, y_predM))

In [ ]:
print(classification_report(y_test, y_predM))